In [2]:
import pandas as pd
import numpy as np
from seaborn import load_dataset
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

Usando o 'diabetes_prediction_dataset' para treinar um modelo e assim obter as probabilidades da classe positiva

In [3]:
df = read_csv("diabetes_prediction_dataset.csv")

Codificando algumas variáveis do Dataset para treinar o classificador

In [4]:
df = pd.get_dummies(df, columns=['gender'], drop_first=True)
df = pd.get_dummies(df, columns=['smoking_history'], drop_first=True)

**Treinando o modelo**
</br>O algoritmo escolhido para treino do modelo é o Random Forest Classifier

In [5]:
y = df['diabetes']
X = df.drop(['diabetes'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((70000, 13), (30000, 13), (70000,), (30000,))

In [7]:
%%time
rf = RandomForestClassifier(n_estimators = 50, max_depth = 5, random_state=42)
rf.fit(X_train, y_train)

CPU times: total: 953 ms
Wall time: 983 ms


RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

Verificando o Classification Report

In [8]:
yhat_train = rf.predict(X_train)
print(classification_report(y_train, yhat_train))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     64047
           1       1.00      0.67      0.80      5953

    accuracy                           0.97     70000
   macro avg       0.99      0.84      0.89     70000
weighted avg       0.97      0.97      0.97     70000



**Calculado o PSI entre o split de treino e o split de teste do modelo**

In [12]:
train_proba = pd.DataFrame({'Probabilidade_Treino': rf.predict_proba(X_train)[:, 1]})
train_proba

,Probabilidade_Treino
0,0.015045
1,0.041227
2,0.015674
3,0.053025
4,0.015043
...,...
69995,0.045895
69996,0.015002
69997,0.019642
69998,0.022137


In [15]:
test_proba = pd.DataFrame({'Probabilidade_Teste' : rf.predict_proba(X_test)[:, 1]})
test_proba

,Probabilidade_Teste
0,0.019820
1,0.012619
2,0.045596
3,0.021220
4,0.048528
...,...
29995,0.768193
29996,0.016443
29997,0.165770
29998,0.064175


Calculando os Decis para as probabilidades de treino e probabilidades de teste

In [20]:
# Usando as probabilidades geradas para a classe na base treino para criar as faixas de decis
decis = [train_proba['Probabilidade_Treino'].quantile(q=x/10) for x in range(1, 10)]

# Função para classificar as probabilidades de acordo com o decil
def atribui_decil(x):
    if x < decis[0]:
        return 1
    elif x < decis[1]:
        return 2
    elif x < decis[2]:
        return 3
    elif x < decis[3]:
        return 4
    elif x < decis[4]:
        return 5
    elif x < decis[5]:
        return 6
    elif x < decis[6]:
        return 7
    elif x < decis[7]:
        return 8
    elif x < decis[8]:
        return 9
    else:
        return 10

In [22]:
train_proba['Decil_Treino'] = train_proba['Probabilidade_Treino'].apply(atribui_decil)
test_proba['Decil_Teste'] = test_proba['Probabilidade_Teste'].apply(atribui_decil)

In [23]:
train_proba

,Probabilidade_Treino,Decil_Treino
0,0.015045,2
1,0.041227,7
2,0.015674,3
3,0.053025,8
4,0.015043,2
...,...,...
69995,0.045895,7
69996,0.015002,2
69997,0.019642,3
69998,0.022137,5


In [24]:
test_proba

,Probabilidade_Teste,Decil_Teste
0,0.019820,4
1,0.012619,1
2,0.045596,7
3,0.021220,4
4,0.048528,7
...,...,...
29995,0.768193,10
29996,0.016443,3
29997,0.165770,10
29998,0.064175,8


In [28]:
count_train = train_proba.groupby(by=['Decil_Treino']).count().rename({'Probabilidade_Treino':'Freq_Treino'}, axis=1)
count_test = test_proba.groupby(by=['Decil_Teste']).count().rename({'Probabilidade_Teste':'Freq_Teste'}, axis=1)
psi = pd.concat([count_train, count_test], axis=1)
psi

,Freq_Treino,Freq_Teste
1,6948,3020
2,7032,3064
3,6693,2852
4,7314,3165
5,7012,3066
6,6982,2979
7,7002,2993
8,7017,2998
9,7000,2930
10,7000,2933


In [29]:
psi['% Treino'] = psi['Freq_Treino'] / psi['Freq_Treino'].sum()
psi['% Teste'] = psi['Freq_Teste'] / psi['Freq_Teste'].sum()
psi['Dif'] = psi['% Treino'] - psi['% Teste']
psi['Ln'] = np.log(psi['% Treino'] / psi['% Teste'])
psi['PSI'] = psi['Dif'] * psi['Ln']
psi

,Freq_Treino,Freq_Teste,% Treino,% Teste,Dif,Ln,PSI
1,6948,3020,0.099257,0.100667,-0.001410,-0.014101,1.987547e-05
2,7032,3064,0.100457,0.102133,-0.001676,-0.016548,2.773752e-05
3,6693,2852,0.095614,0.095067,0.000548,0.005744,3.145437e-06
4,7314,3165,0.104486,0.105500,-0.001014,-0.009661,9.798605e-06
5,7012,3066,0.100171,0.102200,-0.002029,-0.020049,4.067017e-05
6,6982,2979,0.099743,0.099300,0.000443,0.004450,1.970659e-06
7,7002,2993,0.100029,0.099767,0.000262,0.002622,6.866444e-07
8,7017,2998,0.100243,0.099933,0.000310,0.003093,9.572074e-07
9,7000,2930,0.100000,0.097667,0.002333,0.023610,5.508969e-05
10,7000,2933,0.100000,0.097767,0.002233,0.022586,5.044318e-05


In [32]:
print(f"O valor do PSI entre os datasets é {round(psi['PSI'].sum() * 100, 2)}")

O valor do PSI entre os datasets é 0.02


**Referências:**
</br> <a href="https://www.listendata.com/2015/05/population-stability-index.html">POPULATION STABILITY INDEX AND CHARACTERISTIC ANALYSIS - Listen Data</a>
</br> <a href="https://medium.com/model-monitoring-psi/population-stability-index-psi-ab133b0a5d42">Population Stability Index (PSI) - Medium</a>